In [1]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam


2025-08-21 11:16:49.328722: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-08-21 11:16:49.328849: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-08-21 11:16:49.405406: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-08-21 11:16:49.554604: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-08-21 11:16:50.866433: W tensorflow/compiler/tf2

In [2]:
train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)


In [3]:
train_generator = train_datagen.flow_from_directory(
    'dataset/training_set',
    target_size=(224,224),
    batch_size=32,
    class_mode='binary',
    subset='training'
)

Found 6400 images belonging to 2 classes.


In [12]:
test_datagen=ImageDataGenerator(rescale=1./255, validation_split=0.2)
test_generator=test_datagen.flow_from_directory('dataset/test_set',target_size=(224,224),batch_size=32,class_mode='binary',subset='validation'
)

Found 400 images belonging to 2 classes.


In [7]:
base_model = MobileNetV2(weights="imagenet", include_top=False, input_shape=(224,224,3))
base_model.trainable = False  

9406464/9406464 [==============================] - 3s 0us/step


In [8]:
model_mobilenet = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(128, activation='relu'),
    Dense(1, activation='sigmoid')
])

In [9]:
model_mobilenet.compile(optimizer=Adam(1e-4), loss="binary_crossentropy", metrics=["accuracy"])


In [13]:
history_mobilenet = model_mobilenet.fit(train_generator, validation_data=test_generator, epochs=5)


Epoch 1/5


2025-08-21 11:20:22.975923: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 19267584 exceeds 10% of free system memory.
2025-08-21 11:20:26.155554: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 19267584 exceeds 10% of free system memory.
2025-08-21 11:20:26.176599: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 51380224 exceeds 10% of free system memory.
2025-08-21 11:20:26.279169: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 51380224 exceeds 10% of free system memory.
2025-08-21 11:20:26.338643: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 51380224 exceeds 10% of free system memory.


200/200 [==============================] - 257s 1s/step - loss: 0.1348 - accuracy: 0.9542 - val_loss: 0.0697 - val_accuracy: 0.9825
Epoch 2/5
200/200 [==============================] - 253s 1s/step - loss: 0.0571 - accuracy: 0.9800 - val_loss: 0.0613 - val_accuracy: 0.9825
Epoch 3/5
200/200 [==============================] - 253s 1s/step - loss: 0.0461 - accuracy: 0.9837 - val_loss: 0.0573 - val_accuracy: 0.9800
Epoch 4/5
200/200 [==============================] - 271s 1s/step - loss: 0.0388 - accuracy: 0.9869 - val_loss: 0.0532 - val_accuracy: 0.9825
Epoch 5/5
200/200 [==============================] - 234s 1s/step - loss: 0.0324 - accuracy: 0.9892 - val_loss: 0.0565 - val_accuracy: 0.9850


In [27]:
import numpy as np
from keras.preprocessing import image
test_image=image.load_img('dataset/single_prediction/cat_or_dog_1.jpg',target_size=(224,224))
test_image=image.img_to_array(test_image)
test_image=np.expand_dims(test_image,axis=0)
result=model_mobilenet.predict(test_image)
train_generator.class_indices
if result[0][0] > 0.5:
    prediction = "Dog"
else:
    prediction = "Cat"


1/1 [==============================] - 0s 49ms/step


In [28]:
print(prediction)

Dog


In [30]:
# Suppose your model is named `model`
model_mobilenet.save("model/mobileNet_model.h5")


/home/user/.local/lib/python3.10/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
